# GNSS 網形平差

GPS定位中，在任意二個測站上使用GPS衛星接收儀同步觀測成果，可得到二點之間的基線(Base Line)向量觀測值，它是在WGS空間坐標系下的三維坐標差。為了提高定位結果之精度與可靠度，通常需將不同時段觀測的基線向量聯結成網並進行整體平差。由該基線向量構成的網稱為GPS網形，一般採用間接觀測方法進行整體平差。

## 1. 函數模型
GPS網中各待定點的空間直角坐標平差值為參數，該參數之型式定為：

$\begin{bmatrix}\hat{X}_i \\ \hat{Y}_i \\ \hat{Z}_i\end{bmatrix}=
\begin{bmatrix}X^0_i \\ Y^0_i \\ Z^0_i\end{bmatrix}+
\begin{bmatrix}\delta X_i \\ \delta Y_i \\ \delta Z_i\end{bmatrix}
$

若GPS 基線向量觀測值為：
$\begin{equation}(\Delta X_{ij},\Delta Y_{ij},\Delta Z_{ij})\end{equation}$

  $\begin{equation}\Delta X_{ij} = X_j - X_i \end{equation}$

  $\begin{equation}\Delta Y_{ij} = Y_j - Y_i\end{equation}$

  $\begin{equation}\Delta Z_{ij} = Z_j - Z_i\end{equation}$

則三維坐標差，即基線向量觀測值的平差值為：

$\begin{bmatrix}v_{X_{ij}} \\ v_{Y_{ij}} \\ v_{Z_{ij}} \end{bmatrix}=
\begin{bmatrix}\delta x_j \\ \delta x_j \\ \delta z_j\end{bmatrix} -
\begin{bmatrix}\delta x_i \\ \delta x_i \\ \delta z_i\end{bmatrix} +
\begin{bmatrix}X^0_i - X^0_j - \Delta X_{ij} \\ 
Y^0_i - Y^0_j - \Delta Y_{ij} \\
Z^0_i - Z^0_j - \Delta Z_{ij}\end{bmatrix}
$

或
$\begin{bmatrix}v_{X_{ij}} \\ v_{Y_{ij}} \\ v_{Z_{ij}} \end{bmatrix}=
\begin{bmatrix}\delta x_j \\ \delta y_j \\ \delta z_j\end{bmatrix} -
\begin{bmatrix}\delta x_i \\ \delta y_i \\ \delta z_i\end{bmatrix} +
\begin{bmatrix}\Delta X_{ij} - \Delta X^0_{ij} \\ 
\Delta Y_{ij} - \Delta Y^0_{ij} \\
\Delta Z_{ij} - \Delta Z^0_{ij} \end{bmatrix}
$

令
$V_k=\begin{bmatrix}v_{X_{ij}} \\ v_{Y_{ij}} \\ v_{Z_{ij}} \end{bmatrix}
\hat{X}_i=\begin{bmatrix}\delta x_i \\ \delta Y_i \\ \delta Z_i\end{bmatrix}
\hat{X}_j=\begin{bmatrix}\delta X_j \\ \delta Y_j \\ \delta Z_j\end{bmatrix}
\Delta X^0_{ij}=\begin{bmatrix}X^0_j - X^0_i \\ 
Y^0_j - Y^0_i \\
Z^0_j - Z^0_i \end{bmatrix}
\Delta X_{ij}=\begin{bmatrix}\Delta X_{ij}\\ 
\Delta Y_{ij}\\
\Delta Z_{ij}\end{bmatrix}
$

則編號為 k 的基線向量觀測方程式為：
\begin{equation}
V_k
= \hat{X}_j - \hat{X}_i + \Delta X_{ij} - \Delta X^0_{ij} 
= \hat{X}_j - \hat{X}_i + \Delta X_{ij} - (X^0_j - X^0_i)
\end{equation}

當網中有n 個待定點與m 條基線向量時，則GPS 網形之誤差方程式為：
\begin{equation}
V=A \hat{X} - L
\end{equation}


## 2. 隨機模型 (權函數)

隨機模型之一般型式仍為：
$\begin{equation}
\Sigma_{LL}= \sigma^2_0Q = \sigma^2_0p^{-1}
\end{equation}$
或
$\begin{equation}
P=\frac{1}{\sigma^2_0}\Sigma^{-1}_{LL}
\end{equation}$


以二台GPS 接收儀觀測的結果為例，說明GPS 網形平差中隨機模型的
組成。用二台GPS 接收儀進行觀測，在一個時段內只能得到一條觀測基線
向量
$\begin{equation}(\Delta X_{ij},\Delta Y_{ij},\Delta Z_{ij})\end{equation}$
，其中三個觀測坐標分量是相關的，觀測基線向量的
方差－協方差矩陣是由GPS 基線計算軟體計算後得到的，其型式為：

$\begin{equation}
\Sigma_{LL}=\end{equation} 
\begin{bmatrix}
\sigma^2_{\Delta X_{ij}} & 
\sigma_{{\Delta X_{ij}}{\Delta Y_{ij}}} &
\sigma_{{\Delta X_{ij}}{\Delta Z_{ij}}} \\ 
& \sigma^2_{\Delta Y_{ij}}
&\sigma_{{\Delta Y_{ij}}{\Delta Z_{ij}}} \\
& & \sigma^2_{\Delta Z_{ij}}
\end{bmatrix}
$

不同的觀測基線向量之間是互相獨立的，因此，權矩陣 P 是一個3×3
子矩陣所組成的對角陣。

$\begin{equation}
P=Q^{-1}_{LL}=
\frac{1}{\sigma^2_0}\Sigma^{-1}_{LL}
=\frac{1}{\sigma^2_0}
\begin{bmatrix}
p_1 & & \\
& p_2 & \\
& & ... & \\
& & & P_m
\end{bmatrix}
\end{equation}$

## Python GNSS 網形平差最小二乘法模組

GNSS 通用函數

In [0]:
import numpy as np
from numpy.linalg import inv
import math

# General purpose matrix printing function
def Matrix_print(X,matrix_title="Matrix"):
    print("\n%s"%matrix_title)
    
    if len(X.shape)==1:
        for x in X:
            print('%10.4f'%(x))
    else:
        print(X)
        
# General purpose matrix printing function for two vectors
def Matrix_print2(X,sX,matrix_title="Matrix",unit=""):
    print("\n%s"%matrix_title)
    
    for x,sx in zip(X,sX):
        if unit is None:
            print('%.4f +/- %.4f'%(x,sx))
        else:
            print('%.4f%s +/- %.4f (%s)'%(x, unit, sx, unit))

# Compare sting
def compare_names(name,obs_pts):
    id=0
    for i in range(len(obs_pts)):
        if (name==obs_pts[i]):
            id=i
            break
    
    return id

# 由輸入的sigma，組成上三角矩陣
def tri_back(sigma):
    n=len(sigma)
    m=n
    for i in range(n):
        m=m-i
        if (m==0):
            break

    tri=np.zeros([i,i])
    id=0
    id2=0
    for k in range(i,0,-1):
        #print(tri[id,id:id+k])
        tri[id,id:id+k]=sigma[id2:id2+k]
        id=id+1
        id2=id2+k

    return tri

def GNSS_LSEA(A,L,P):
    m,n=A.shape
    #print(m,n)
    
    #% 未知數的答解 
    #% 法方程式之反矩陣
    # N=A'*P*A;
    N=np.dot(A.T,P)
    N=np.dot(N,A)

    # %N_inv=inv(N);
    # U=A'*P*L;
    U=np.dot(A.T,P)
    U=np.dot(U,L)

    N_inv=inv(N)
    X=np.dot(N_inv,U)

    #% 誤差分析
    #  V=A*X-L;
    V=np.dot(A,X)-L

    #%單位權標準誤差   
    #sigma0=sqrt((V'*P*V)./(m-n));
    q=np.dot(V.T,P)
    q=np.dot(q,V)
    sigma0=math.sqrt(q/(m-n))

    #%未知數精度矩陣
    DX=N_inv*(sigma0**2)
    sX=np.diag(DX)**0.5

    #%觀測量精度分析   
    DL=np.dot(A,N_inv)
    DL=sigma0**2*np.dot(DL,A.T)
    sL=(np.diag(DL))**0.5

    return X, V, sigma0, DX, sL, P, N, U

### GNSSNet物件類別定義

In [0]:
# GNSS_net клеє
class GNSS_net:
    
  # private variables
  
  # private methods
  #self.__GNSS_design_matrix()
  
  def __init__(self):
      self._ProjectName='[Empty GNSS Net Project]'
      self._num_control_pts = 0
      self._num_obs_pts = 0
      self._num_obs_baselines = 0
      self._ctrl_pts=[]
      self._obs_pts=[]
      self._pts=[]
      self._baselines=[]
      self._pts_name=[]
      self._A=np.zeros([1,1])
      self._L=np.zeros([1,1])
      self._P=np.zeros([1,1])
      self._sigma0=0
        
  # 物件類別方法  
  def GNSS_read_Adat(self, path, sigma0, ComputeObs=False):
      self.__read_Adat(path, sigma0, ComputeObs)
      
  def GNSS_design_matrix(self):
      self.__GNSS_design_matrix()

  def GNSS_Report(self,X, V, sigma0, DX, sL, N, U):
      self.__GNSS_Report(X, V, sigma0, DX, sL, N, U)
  
  # 物件類別屬性
  # ProjectName
  @property
  def ProjectName(self):
      return self._ProjectName
  @ProjectName.setter
  def ProjectName(self, ProjectName):
      self._ProjectName = ProjectName

  # num_control_pts
  @property
  def num_control_pts(self):
      return self._num_control_pts
  @num_control_pts.setter
  def num_control_pts(self,num_control_pts):
      self._num_control_pts = num_control_pts

  # num_obs_pts
  @property
  def num_obs_pts(self):
      return self._num_obs_pts
  @num_obs_pts.setter
  def num_obs_pts(self,num_obs_pts):
      self._num_obs_pts = num_obs_pts

  # num_obs_baselines
  @property
  def num_obs_baselines(self):
      return self._num_obs_baselines
  @num_obs_baselines.setter
  def num_obs_baselines(self,num_obs_baselines):
      self._num_obs_baselines = num_obs_baselines
      
  @property
  def ctrl_pts(self):
      return self._ctrl_pts
  
  @property
  def baselines(self):
      return self._baselines
  
  @property
  def obs_pts(self):
      return self._obs_pts

  @property
  def sigma0(self):
      return self._sigma0
  
  @property
  def A(self):
      return self._A
  
  @property
  def L(self):
      return self._L
  
  @property
  def P(self):
      return self._P
  
  # Setup GNSS_design_matrix according to input data.
  def __GNSS_design_matrix(self):
      m = self._num_obs_pts*3
      n = self._num_obs_baselines*3
      k = self._num_control_pts + self._num_obs_pts
      #print(m,n,k)

      self._A=np.zeros([n,m])
      self._L=np.zeros([n,1])
      self._P=np.zeros([n,n])

      id=0
      for i in range(0,n,3):
          sigma=self._baselines[id][7] 
          sigma=inv(np.dot(sigma,1.0/self._sigma0**2))
          self._P[i:i+3,i:i+3]=sigma
          
          id1=self._baselines[id][0]
          id2=self._baselines[id][1]
          Fid=(id1 - self._num_control_pts)*3
          Tid=(id2 - self._num_control_pts)*3

          #print(id1,id2,Fid,Tid)
          if (id1 >= self._num_control_pts):
              self._A[i][Fid]=-1
              self._A[i+1][Fid+1]=-1
              self._A[i+2][Fid+2]=-1

          if (id2 >= self._num_control_pts):
              self._A[i][Tid]=1
              self._A[i+1][Tid+1]=1
              self._A[i+2][Tid+2]=1
          
          FromID = self._baselines[id][0]
          ToID   = self._baselines[id][1]
          
          X1=self._pts[FromID][2]
          Y1=self._pts[FromID][3]
          Z1=self._pts[FromID][4]
          X2=self._pts[ToID][2]
          Y2=self._pts[ToID][3]
          Z2=self._pts[ToID][4]
          
          DX=X2-X1
          DY=Y2-Y1
          DZ=Z2-Z1
          self._L[i,0]  = self._baselines[id][4]-DX        
          self._L[i+1,0]= self._baselines[id][5]-DY
          self._L[i+2,0]= self._baselines[id][6]-DZ
          id=id+1                    
          

  # initial a GNSS Network from Aadj file.
  def __read_Adat(self, path,sigma0,ComputeObs):
      self._sigma0=sigma0
      self._pts_name=[]
      f=open(path, "r")
      self._ProjectName = f.readline()
      items=f.readline().split(' ')
      
      # Load Project infomation: ProjectName
      self._num_control_pts = int(items[0])
      self._num_obs_pts = int(items[1])
      self._num_obs_baselines = int(items[2])
      
      self._ctrl_pts=[]
      # Load control points.
      # Point ID start from 0
      for i in range(self._num_control_pts):
          items=f.readline().split(' ')
          ID   = i
          name = items[0]
          X    = float(items[1])
          Y    = float(items[2])
          Z    = float(items[3])
          ctrl = [ID, name, X, Y, Z]
          self._ctrl_pts.append(ctrl)
          self._pts.append(ctrl)
          self._pts_name.append(name)
      
      # load baselines
      self._baselines=[]
      for i in range(self._num_obs_baselines):
          items   = f.readline().split(' ')
          FromID  = 0
          ToID    = 0
          fromName= items[0]
          toName  = items[1]
          dX      = float(items[2])
          dY      = float(items[3])
          dZ      = float(items[4])
          sigma   = np.zeros(6)
          for j in range(5,11):
              sigma[j-5]=float(items[j])
          
          # put sigma data as upper triangle matrix
          sigma_tri = tri_back(sigma)
          sigma_full= sigma_tri+sigma_tri.T
          for i in range(3):
              sigma_full[i,i]=sigma_full[i,i]/2.0
              
          baseline=[FromID,ToID,fromName,toName,dX,dY,dZ,sigma_full]
          self._baselines.append(baseline)
      
      print(self._baselines)
      # load initial observation points.
      # Observation ID start after number of control points.
      self._obs_pts=[]
      if (ComputeObs==False):
          for i in range(self._num_obs_pts):
              items=f.readline().split(' ')
              ID   = self._num_control_pts + i
              name = items[0]
              X    = float(items[1])
              Y    = float(items[2])
              Z    = float(items[3])
              obs  = [ID, name, X, Y, Z]
              self._obs_pts.append(obs)
              self._pts.append(obs)
              self._pts_name.append(name)
      else:
          # 初始化未知測站概略座標
          for i in range(self._num_obs_pts):
                items=f.readline().split(' ')
                ID   = self._num_control_pts + i
                name = items[0]
                X    = float(items[1])
                Y    = float(items[2])
                Z    = float(items[3])
                obs  = [ID, name, X, Y, Z]
                self._obs_pts.append(obs)
                self._pts.append(obs)
                self._pts_name.append(name)
          
      f.close()
      # 把base_lines裡面的測站名稱& ID放進obs_pts內
      pts_name=self._pts_name

      for i in range(self._num_obs_baselines):
          FromName = self._baselines[i][2]
          ToName   = self._baselines[i][3]
          self._baselines[i][0]=compare_names(FromName, pts_name)
          self._baselines[i][1]=compare_names(ToName, pts_name)
      
      if (ComputeObs==True):
        # 用已知值估算未知測站概略座標
        for i in range(self._num_obs_baselines):
            id1=self._baselines[i][0]
            id2=self._baselines[i][1]
            #print(i,id1,id2,self._pts[id1][2],self._pts[id2][2])
            if (id1 >= self._num_control_pts):
              if (self._pts[id1][2]==0):
                  self._pts[id1][1]=self._baselines[i][2]
                  self._pts[id1][2]=self._pts[id2][2]-self._baselines[i][4]
                  self._pts[id1][3]=self._pts[id2][3]-self._baselines[i][5]
                  self._pts[id1][4]=self._pts[id2][4]-self._baselines[i][6]
                  #print(self._pts[id1])
                    
            if (id2 >= self._num_control_pts):    
              if (self._pts[id2][2]==0):
                  self._pts[id2][1]=self._baselines[i][3]
                  self._pts[id2][2]=self._pts[id1][2]+self._baselines[i][4]
                  self._pts[id2][3]=self._pts[id1][3]+self._baselines[i][5]
                  self._pts[id2][4]=self._pts[id1][4]+self._baselines[i][6]
                  #print(self._pts[id2])
            
        for i in range(self._num_obs_pts):
            k=i+self._num_control_pts
            self._obs_pts[i][2]=self._pts[k][2]
            self._obs_pts[i][3]=self._pts[k][3]
            self._obs_pts[i][4]=self._pts[k][4]
        #print(self._obs_pts)
          
  def __GNSS_Report(self,X, V, sigma0, DX, sL, N, U):
      print('***** GNSS 最小二乘法 網形平差報表 *****')

      print('\nGNSS Net專案名稱: %s'%(self._ProjectName))

      print('\n控制點數: %d'%(self._num_control_pts))
      print('\n控制點座標')
      for i in range(self._num_control_pts):
          ctrl=self._ctrl_pts[i]
          print('%d %s\t%14.5f\t%14.5f\t%14.5f'%(ctrl[0],ctrl[1],ctrl[2],ctrl[3],ctrl[4]))

      print('\n觀測點數: %d'%(self._num_obs_pts))
      print('\n觀測點概略座標')
      for i in range(self._num_obs_pts):
          obs=self._obs_pts[i]
          print('%d %s\t%14.5f\t%14.5f\t%14.5f'%(obs[0],obs[1],obs[2],obs[3],obs[4]))

      print('\n觀測基線數: %d'%(self._num_obs_baselines))
      print('\n基線觀測資料')
      for i in range(self._num_obs_baselines):
          BSL=self._baselines[i]
          print('%s(%d)  ->  %s(%d)\t%12.4f\t%12.4f\t%12.4f'%
                (BSL[2],BSL[0],BSL[3],BSL[1],BSL[4],BSL[5],BSL[6]))

      Matrix_print(self._A,'設計矩陣A')
      Matrix_print(self._L,'觀測矩陣L')

      Matrix_print(self._P,'加權矩陣P')
      Matrix_print(N,'法方程式 N矩陣')

      print('\n單位權標準誤差：+/- %.5f'%(sigma0) )

      sX=np.diag(DX)**0.5
      print('\n控制點及觀測點座標')
      for i in range(self._num_control_pts):
          ctrl=self._ctrl_pts[i]
          print('%d %s %14.5f %14.5f %14.5f'%(ctrl[0],ctrl[1],ctrl[2],ctrl[3],ctrl[4]))

      id=0    
      for i in range(0,self._num_obs_pts*3,3):
          obs=self._obs_pts[id]
          print('%d %s %14.5f %14.5f %14.5f %6.5f %6.5f %6.5f'
                %(obs[0],obs[1],obs[2]+X[i],obs[3]+X[i+1],obs[4]+X[i+2],
                  sX[i],sX[i+1],sX[i+2]))
          id+=1
      print('\n基線平差結果與精度分析')
      id=0    
      for i in range(0,self._num_obs_baselines*3,3):
          BSL=self._baselines[id]
          dx=BSL[4]+V[i]
          dy=BSL[5]+V[i+1]
          dz=BSL[6]+V[i+2]
          print('%s(%d) -> %s (%d) %12.5f %12.5f %12.5f %6.5f %6.5f %6.5f'
                %(BSL[2],BSL[0],BSL[3],BSL[1],dx,dy,dz,sL[i],sL[i+1],sL[i+2]))
          id+=1
      #Matrix_print2(X,sX,'未知數及精度分析',unit='m')
      #Matrix_print(sL,'觀測量精度矩陣')
      Matrix_print(U,'法方程式 U矩陣')
      Matrix_print(X,'未知數X')
      Matrix_print(V,'誤差矩陣V')
      

### 範例
<img src='https://drive.google.com/uc?id=1d2g0QpjfGRF7xpn98Z3ijBV1jo4gmwD4'>

### 呼叫GNSSNet物件類別

1. 初始化物件類別
2. 從GNSS輸入檔案(.Adat)格式輸入GNSS網形

In [7]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
data_dir='drive/My Drive/Colab Notebooks/Labs/Lab4_Adjustment/'
gnss_file=data_dir+'ex178.Adat'
psigma0=1

# 建立一個新的GNSS網形平差物件
wolf_gnss_net=GNSS_net()

# ComputeObs=True代表要由程式自動計算概略座標
wolf_gnss_net.GNSS_read_Adat(gnss_file,psigma0, ComputeObs=True)

[[0, 0, 'A', 'C', 11644.2232, 3601.2165, 3399.255, array([[ 9.884e-04, -9.580e-06,  9.520e-06],
       [-9.580e-06,  9.377e-04, -9.520e-06],
       [ 9.520e-06, -9.520e-06,  9.827e-04]])], [0, 0, 'A', 'E', -5321.7164, 3634.0754, 3173.6652, array([[ 2.158e-04, -2.100e-06,  2.160e-06],
       [-2.100e-06,  1.919e-04, -2.100e-06],
       [ 2.160e-06, -2.100e-06,  2.005e-04]])], [0, 0, 'B', 'C', 3960.5442, -6681.2467, -7279.0148, array([[ 2.305e-04, -2.230e-06,  2.070e-06],
       [-2.230e-06,  2.546e-04, -2.230e-06],
       [ 2.070e-06, -2.230e-06,  2.252e-04]])], [0, 0, 'B', 'D', -11167.6076, -394.5204, -907.9593, array([[ 2.700e-04, -2.750e-06,  2.850e-06],
       [-2.750e-06,  2.721e-04, -2.720e-06],
       [ 2.850e-06, -2.720e-06,  2.670e-04]])], [0, 0, 'D', 'C', 15128.1647, -6286.7054, -6371.0583, array([[ 1.461e-04, -1.430e-06,  1.340e-06],
       [-1.430e-06,  1.614e-04, -1.440e-06],
       [ 1.340e-06, -1.440e-06,  1.308e-04]])], [0, 0, 'D', 'E', -1837.7459, -6253.8534, -6596.6697

## 使用GNSSNet物件類別內的方法

1. GNSS_design_matrix設計矩陣
2. GNSS_LSEA最小二乘法模組 (外部方法)
3. GNSS_Report產製報表

In [14]:
# 組成設計矩陣
wolf_gnss_net.GNSS_design_matrix()

# 最小二乘法
X, V, sigma0, DX, sL, P, N, U = GNSS_LSEA(wolf_gnss_net.A, wolf_gnss_net.L,wolf_gnss_net.P)

wolf_gnss_net.GNSS_Report(X, V, sigma0, DX, sL, N, U)

***** GNSS 最小二乘法 網形平差報表 *****

GNSS Net專案名稱: Example 17.8


控制點數: 2

控制點座標
0 A	     402.35087	-4652995.30109	 4349760.77753
1 B	    8086.03178	-4642712.84739	 4360439.08326

觀測點數: 4

觀測點概略座標
2 C	   12046.57407	-4649394.08459	 4353160.03253
3 D	   -3081.57582	-4643107.36779	 4359531.12396
4 E	   -4919.36553	-4649361.22569	 4352934.44273
5 F	    1518.80317	-4648399.14009	 4354116.68373

觀測基線數: 14

基線觀測資料
A(0)  ->  C(2)	  11644.2232	   3601.2165	   3399.2550
A(0)  ->  E(4)	  -5321.7164	   3634.0754	   3173.6652
B(1)  ->  C(2)	   3960.5442	  -6681.2467	  -7279.0148
B(1)  ->  D(3)	 -11167.6076	   -394.5204	   -907.9593
D(3)  ->  C(2)	  15128.1647	  -6286.7054	  -6371.0583
D(3)  ->  E(4)	  -1837.7459	  -6253.8534	  -6596.6697
F(5)  ->  A(0)	  -1116.4523	  -4596.1610	  -4355.9062
F(5)  ->  C(2)	  10527.7852	   -994.9377	   -956.6246
F(5)  ->  E(4)	  -6438.1364	   -962.0694	  -1182.2305
F(5)  ->  D(3)	  -4600.3787	   5291.7785	   5414.4311
F(5)  ->  B(1)	   6567.2311	   5686.2926	   6322.3917


In [15]:
data_dir='drive/My Drive/Colab Notebooks/Labs/Lab4_Adjustment/'
gnss_file=data_dir+'ex13.Adat'
psigma0=0.00298

# 建立一個新的GNSS網形平差物件
wolf_gnss_net=GNSS_net()

# 從給定的資料讀取資料
wolf_gnss_net.GNSS_read_Adat(gnss_file,psigma0)

# 組成設計矩陣
wolf_gnss_net.GNSS_design_matrix()

# 最小二乘法
X, V, sigma0, DX, sL, P, N, U = GNSS_LSEA(wolf_gnss_net.A, wolf_gnss_net.L,wolf_gnss_net.P)

wolf_gnss_net.GNSS_Report(X, V, sigma0, DX, sL, N, U)

[[0, 0, 'B', 'A', -1218.561, -1039.227, 1737.72, array([[ 2.320999e-07, -5.097008e-07, -4.371401e-07],
       [-5.097008e-07,  1.339931e-06,  1.109356e-06],
       [-4.371401e-07,  1.109356e-06,  1.008592e-06]])], [0, 0, 'D', 'A', 270.457, -503.208, 1879.923, array([[ 1.004894e-06, -2.396533e-06, -2.319683e-06],
       [-2.396533e-06,  6.341291e-06,  5.902876e-06],
       [-2.319683e-06,  5.902876e-06,  6.035577e-06]])], [0, 0, 'D', 'B', 1489.013, 536.03, 142.218, array([[ 5.850064e-07, -1.329620e-06, -1.252374e-06],
       [-1.329620e-06,  3.362548e-06,  3.069820e-06],
       [-1.252374e-06,  3.069820e-06,  3.019233e-06]])], [0, 0, 'C', 'B', 1405.531, -178.157, 1171.38, array([[ 1.205319e-06, -2.636702e-06, -2.174106e-06],
       [-2.636702e-06,  6.858585e-06,  5.480745e-06],
       [-2.174106e-06,  5.480745e-06,  4.820125e-06]])], [0, 0, 'D', 'C', 83.497, 714.153, -1029.199, array([[ 9.662657e-06, -2.175476e-05, -1.971468e-05],
       [-2.175476e-05,  5.194777e-05,  4.633565e-05],
  